# Vorbereitungen

In [ ]:
!pip install openai pydantic

In [ ]:
# OPENAI KEY lesen
import os
try:
    from google.colab import userdata
    OPENAI_KEY = userdata.get('OPENAI_KEY')
except:
    OPENAI_KEY = os.getenv('OPENAI_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_KEY


# Function Calling

https://platform.openai.com/docs/guides/function-calling/supported-models

Not all model versions are trained with function calling data. Function calling is supported with the following models: `gpt-4`, `gpt-4-turbo-preview`, `gpt-4-0125-preview`, `gpt-4-1106-preview`, `gpt-4-0613`, `gpt-3.5-turbo`, `gpt-3.5-turbo-0125`, `gpt-3.5-turbo-1106`, and `gpt-3.5-turbo-0613`.

In addition, parallel function calls is supported on the following models: `gpt-4-turbo-preview`, `gpt-4-0125-preview`, `gpt-4-1106-preview`, `gpt-3.5-turbo-0125`, and `gpt-3.5-turbo-1106`.

In [ ]:
from openai import OpenAI
from pydantic import BaseModel, Field
import json

In [ ]:
client = OpenAI()

Aus folgendem Text sollen alle vorkommenden Personen mit Vorname + Name extrahiert werden

In [ ]:
#https://www.spiegel.de/sport/fussball-bundesliga-erst-fliegen-suessigkeiten-dann-besiegt-bayer-leverkusen-bayern-muenchen-a-16b38b4f-2ab7-4d78-8b15-e63e157e1fad#
text = '''Leihspieler aus München trifft zur Bayer-Führung

Als dann Fußball gespielt wurde, präsentierten sich die Hausherren gefährlicher: Nationalspieler Robert Andrich spielte von der linken Seite flach 
nach innen, dort gelang Josip Stanišić per Direktabnahme das 1:0 (18. Minute). Ausgerechnet Stanišić: Der Verteidiger war zu Beginn dieser Saison als 
Leihgabe von den Münchnern gekommen.

Und es wurde noch besser für die Leverkusener: Kurz nach der Halbzeit gelang Bayer das zweite Tor der Partie. Alejandro Grimaldo traf nach einem 
Pass von Nathan Tella sehenswert aus spitzem Winkel an Bayern-Keeper Manuel Neuer vorbei ins Tor (50.).

Die Münchner versuchten nun, den Druck zu erhöhen – blieben aber harmlos. Leverkusen verteidigte konsequent und traf sogar noch je einmal die Latte 
(von der Eckfahne) und den Pfosten. Jeremie Frimpong sorgte mit einem Schuss ins leere Tor in der Nachspielzeit für den Schlusspunkt. Neuer war bei 
einer Bayern-Ecke in den gegnerischen Strafraum gelaufen.'''

Mit `pydantic` können wir das Datenschema definieren welches für die Extraktion verwendet wrid.

https://docs.pydantic.dev/1.10/usage/schema/#__tabbed_2_2

## Personen extrahieren

In [ ]:
class Person(BaseModel):
    vorname: str
    nachname: str

class PersonenListe(BaseModel):
    personen: list[Person]

Die Funktionsdefinition hat die folgenden Felder:   
   
`name`: Name der Funktion   
`description`: Beschreibung der Funktion - diese ist wichtig, damit das Modell versteht, welche Fähigkeiten die Funktion hat und wann sie aufgerufen werden soll.   
`parameters`: Pydantic Schema   

In [ ]:
extract_people_function = {
        "type": "function",
        "function": {
                "name": "extract_people_list",
                "description": "Extract alle Personen die im Text genannt werden",
                "parameters": PersonenListe.model_json_schema()
        }
}

In [ ]:
PersonenListe.model_json_schema()

Die Funtkion(en) müssen dem API-Call mit dem `tools` Parameter übergeben werden.

In [ ]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  tools=[extract_people_function],
  tool_choice="auto",
  messages=[
    {"role": "system", "content": "Extrahiere alle Personen die im Text genannt werden"},
    {"role": "user", "content": text}
  ]
)

In [ ]:
data = completion.choices[0].message.tool_calls[0].function.arguments
json.loads(data)

## Vereine extrahieren

In [ ]:
class FussballMannschaft(BaseModel):
    mannschafts_name: str

class FussballMannschaftListe(BaseModel):
    mannschaften: list[FussballMannschaft]

In [ ]:
extract_clubs_function = {
        "type": "function",
        "function": {
                "name": "extract_club_list",
                "description": "Extrahiere alle Fussballmannschaften die im Text genannt werden",
                "parameters": FussballMannschaftListe.model_json_schema()
        }
}

In [ ]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  tools=[extract_clubs_function],
  tool_choice="auto",
  messages=[
    {"role": "system", "content": "Extrahiere alle Fussballmannschaften die im Text genannt werden."},
    {"role": "user", "content": text}
  ]
)

In [ ]:
data = completion.choices[0].message.tool_calls[0].function.arguments
json.loads(data)

## Personen + Vereine gleichzeitig extrahieren

In [ ]:
completion = client.chat.completions.create(
  model="gpt-4-0125-preview",
  tools=[extract_people_function, extract_clubs_function],
  tool_choice="auto",
  messages=[
    {"role": "system", "content": "Extrahiere alle Personen und Fussballmannschaften die im Text genannt werden"},
    {"role": "user", "content": text}
  ]
)

In [ ]:
data = completion.choices[0].message.tool_calls[0].function.arguments
json.loads(data)

In [ ]:
data = completion.choices[0].message.tool_calls[1].function.arguments
json.loads(data)